## Cointab Assignment Report

### Name: Saliya Bano Sheikh
### contact: sbsheikh98@gmail.com

Cointab Software Private Limited

Data Analyst - Assignment

#### Overview
Please understand the below mentioned real-life scenario and try to solve the assignment.

The sample data is attached in the link provided below for your reference.

#### Business Scenario
You are a data analyst and your client has a large ecommerce company in India (let’s call it X).
X gets a thousand orders via their website on a daily basis and they have to deliver them as fast
as they can. For delivering the goods ordered by the customers, X has tied up with multiple
courier companies in India as delivery partners who charge them some amount per delivery.

#### The charges are dependent upon two factors:
1. Weight of the product and the Price of the product.
2. Distance between the warehouse (pickup location) and customer’s delivery address
(destination location)
On an average, the delivery charges are Rs. 100 per shipment. So if X ships 1,00,000 orders
per month, they have to pay approximately Rs. 1 crore to the courier companies on a monthly
basis as charges.
As the amount that X has to pay to the courier companies is very high, they want to verify if the
charges levied by their Delivery partners per Order are correct.

#### Input Data
Left Hand Side (LHS) Data (X’s internal data spread across three reports)

● Website order report- which will list Order IDs ,various products (SKUs) part of each
order and the Payment Type of the Order ( COD or Prepaid). Order ID is common
identifier between X’s order report and courier company invoice

● Warehouse pincode to All India pincode mapping -(this should be used to figure out
delivery zone (a/b/c/d/e) and during analysis compare against one reported by courier
company in their CSV invoice per Order ID

● SKU master with gross weight of each product. This should be used to calculate total
weight of each order and during analysis compare against one reported by courier
company in their CSV invoice per Order ID. The courier company calculates weight in
slabs that is applicable for that delivery zone, so first you have to figure out the total
weight of the shipment and then figure out applicable weight (based on zone’s weight
slab).

For example:

Total Weight Applicable Zone Applicable Weight

Slab (based on zone)

Applicable Weight
400gm a 0.25 0.5

400gm b 0.5 0.5

400gm c 0.75 0.75

2.2KG a 0.25 2.25

2.2KG b 0.5 2.5

2.2KG c 0.75 2.25

RHS Data (courier company invoice in CSV file)

● Invoice in CSV file mentioning AWB Number (courier company’s own internal ID), Order
ID (company X’s order ID), weight of shipment, warehouse pickup pincode, customer
delivery pincode, zone of delivery, charges per shipment, type of shipment

● Courier charges rate card at weight slab and pincode level. If the invoice mentions
“Forward charges” then only forward charges (“fwd”) should be applicable as per zone
and fixed & additional weights based on weight slabs. If the invoice mentions “Forward
and rto charges” then forward charges (“fwd”) and RTO charges (“rto”) should be
applicable as per zone and fixed & additional weights based on weight slabs.

● For the first slab of that zone, “fixed” rate as per the slab is applicable. For each
additional slab, “additional” weight in the same proportion is applicable. Total charges will
be “fixed” + “total additional” if any. For example: weight 2.2KG, Zone C. So for Zone C
the slab length = 0.75KG. So the total applicable weight = 2.25KG. For the first 0.75 KG
the charge is “fwd”, and for each 0.75 after the first, charges will be additional charges.

● Once this is done, then you need to calculate the COD charges. For this case, if the
order is Prepaid then COD charge is 0 and if the order is COD then we need to check
the amount of the order
If the order amount is less than equal to 300 then COD charge is 15
Else the COD charge is 5% of the Order Amount

● Total Charges = Total forward charge + Total RTO charge + COD charge.
Output Data 1
Create a resultant CSV/Excel file with the following columns:

● Order ID

● AWB Number

● Total weight as per X (KG)

● Weight slab as per X (KG)

● Total weight as per Courier Company (KG)

● Weight slab charged by Courier Company (KG)


● Delivery Zone as per X

● Delivery Zone charged by Courier Company

● Expected Charge as per X (Rs.)

● Charges Billed by Courier Company (Rs.)


● Difference Between Expected Charges and Billed Charges (Rs.)

Output Data 2

Create a summary table

Count Amount (Rs.)
Total orders where X has been correctly charged <count> <total invoice
amount>
Total Orders where X has been overcharged <count> <total
overcharging
amount>
Total Orders where X has been undercharged <count> <total
undercharging
amount>
 


## Importing libraries

In [1]:
# Import libraries

import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

# pd.set_option('display.max_rows',None)

## Importing dataset

### company_order_report

In [2]:
company_order_report = pd.read_excel("Company X - Order Report.xlsx")
company_order_report


,ExternOrderNo,SKU,Order Qty,Payment Mode,Item Price(Per Qty.)
0,2001827036,8904223818706,1,COD,233
1,2001827036,8904223819093,1,COD,233
2,2001827036,8904223819109,1,COD,233
3,2001827036,8904223818430,1,COD,470
4,2001827036,8904223819277,1,COD,233
...,...,...,...,...,...
395,2001806229,8904223818942,1,COD,286
396,2001806229,8904223818850,1,COD,563
397,2001806226,8904223818850,2,COD,1125
398,2001806210,8904223816214,1,COD,402


### company_invoice

In [3]:
company_invoice = pd.read_excel("Courier Company - Invoice.xlsx")
company_invoice

,AWB Code,Order ID,Charged Weight,Warehouse Pincode,Customer Pincode,Zone,Type of Shipment,Billing Amount (Rs.)
0,1091117222124,2001806232,1.30,121003,507101,d,Forward charges,140.0
1,1091117222194,2001806273,1.00,121003,486886,d,Forward charges,101.2
2,1091117222931,2001806408,2.50,121003,532484,d,Forward charges,224.6
3,1091117223244,2001806458,1.00,121003,143001,b,Forward charges,61.3
4,1091117229345,2001807012,0.15,121003,515591,d,Forward charges,45.4
...,...,...,...,...,...,...,...,...
119,1091118551656,2001812941,0.73,121003,325207,d,Forward charges,95.2
120,1091117614452,2001809383,0.50,121003,303702,d,Forward and RTO charges,86.7
121,1091120922803,2001820978,0.50,121003,313301,d,Forward charges,45.4
122,1091121844806,2001811475,0.50,121003,173212,b,Forward charges,47.0


### company_master

In [4]:
company_master = pd.read_excel("Company X - SKU Master.xlsx")
company_master

,SKU,Weight (g)
0,8904223815682,210
1,8904223815859,165
2,8904223815866,113
3,8904223815873,65
4,8904223816214,120
...,...,...
61,8904223819505,210
62,8904223819499,210
63,8904223819512,210
64,8904223819543,300


### courier_company_rates

In [5]:
courier_company_rates = pd.read_excel("Courier Company - Rates.xlsx")
courier_company_rates 

,Zone,Weight Slabs,Forward Fixed Charge,Forward Additional Weight Slab Charge,RTO Fixed Charge,RTO Additional Weight Slab Charge
0,A,0.25,29.5,23.6,13.6,23.6
1,B,0.50,33.0,28.3,20.5,28.3
2,C,0.75,40.1,38.9,31.9,38.9
3,D,1.25,45.4,44.8,41.3,44.8
4,E,1.50,56.6,55.5,50.7,55.5


### company_zones

In [6]:
company_zones = pd.read_excel("Company X - Pincode Zones.xlsx")
company_zones

,Warehouse Pincode,Customer Pincode,Zone
0,121003,507101,d
1,121003,486886,d
2,121003,532484,d
3,121003,143001,b
4,121003,515591,d
...,...,...,...
119,121003,325207,b
120,121003,303702,b
121,121003,313301,b
122,121003,173212,e


### renaming the column zone as zone_x 

In [7]:
company_zones.rename(columns = {'Zone':'zone_X'}, inplace = True)
company_zones

,Warehouse Pincode,Customer Pincode,zone_X
0,121003,507101,d
1,121003,486886,d
2,121003,532484,d
3,121003,143001,b
4,121003,515591,d
...,...,...,...
119,121003,325207,b
120,121003,303702,b
121,121003,313301,b
122,121003,173212,e


### Merging SKU_Master and Order_report Datasets 

In [8]:
sku_weight = pd.merge(company_master , company_order_report, on = 'SKU',how="outer" )
sku_weight

,SKU,Weight (g),ExternOrderNo,Order Qty,Payment Mode,Item Price(Per Qty.)
0,8904223815682,210,2001808801,1,Prepaid,965
1,8904223815859,165,2001820978,1,Prepaid,951
2,8904223815859,165,2001811363,1,Prepaid,951
3,8904223815859,165,2001811363,1,Prepaid,951
4,8904223815859,165,2001808801,1,Prepaid,951
...,...,...,...,...,...,...
396,8904223819512,210,2001806733,1,Prepaid,534
397,8904223819543,300,2001811306,1,Prepaid,653
398,SACHETS001,10,2001827036,1,COD,350
399,SACHETS001,10,2001825261,1,Prepaid,350


### Calulating the total weight of the order by its total quantity

In [9]:
sku_weight['weight_by_quantity'] = sku_weight['Weight (g)'] * sku_weight['Order Qty']
sku_weight

,SKU,Weight (g),ExternOrderNo,Order Qty,Payment Mode,Item Price(Per Qty.),weight_by_quantity
0,8904223815682,210,2001808801,1,Prepaid,965,210
1,8904223815859,165,2001820978,1,Prepaid,951,165
2,8904223815859,165,2001811363,1,Prepaid,951,165
3,8904223815859,165,2001811363,1,Prepaid,951,165
4,8904223815859,165,2001808801,1,Prepaid,951,165
...,...,...,...,...,...,...,...
396,8904223819512,210,2001806733,1,Prepaid,534,210
397,8904223819543,300,2001811306,1,Prepaid,653,300
398,SACHETS001,10,2001827036,1,COD,350,10
399,SACHETS001,10,2001825261,1,Prepaid,350,10


### Total weight as per x


In [10]:
sku_weight

,SKU,Weight (g),ExternOrderNo,Order Qty,Payment Mode,Item Price(Per Qty.),weight_by_quantity
0,8904223815682,210,2001808801,1,Prepaid,965,210
1,8904223815859,165,2001820978,1,Prepaid,951,165
2,8904223815859,165,2001811363,1,Prepaid,951,165
3,8904223815859,165,2001811363,1,Prepaid,951,165
4,8904223815859,165,2001808801,1,Prepaid,951,165
...,...,...,...,...,...,...,...
396,8904223819512,210,2001806733,1,Prepaid,534,210
397,8904223819543,300,2001811306,1,Prepaid,653,300
398,SACHETS001,10,2001827036,1,COD,350,10
399,SACHETS001,10,2001825261,1,Prepaid,350,10


In [11]:
tmp = sku_weight.groupby('ExternOrderNo')['weight_by_quantity'].sum()
tmp = tmp.reset_index()

### Remaning the weitht_by_quantity column 


In [12]:
tmp.rename(columns = {'weight_by_quantity':'total_wt_X'}, inplace = True)
tmp

,ExternOrderNo,total_wt_X
0,2001806210,220
1,2001806226,480
2,2001806229,500
3,2001806232,1302
4,2001806233,245
...,...,...
119,2001821995,477
120,2001822466,1376
121,2001823564,672
122,2001825261,1557


### weight slab as per x

In [13]:
tmp['d_500'] = tmp['total_wt_X']/500
tmp['d_500'] = np.ceil(tmp['d_500'])
tmp['weight_slab_X'] = tmp['d_500'] * 0.5
tmp

,ExternOrderNo,total_wt_X,d_500,weight_slab_X
0,2001806210,220,1.0,0.5
1,2001806226,480,1.0,0.5
2,2001806229,500,1.0,0.5
3,2001806232,1302,3.0,1.5
4,2001806233,245,1.0,0.5
...,...,...,...,...
119,2001821995,477,1.0,0.5
120,2001822466,1376,3.0,1.5
121,2001823564,672,2.0,1.0
122,2001825261,1557,4.0,2.0


### Remaning the ExternalOrderNo to Order_ID 


In [14]:
tmp.rename(columns = {'ExternOrderNo':'Order ID'}, inplace = True)
tmp

,Order ID,total_wt_X,d_500,weight_slab_X
0,2001806210,220,1.0,0.5
1,2001806226,480,1.0,0.5
2,2001806229,500,1.0,0.5
3,2001806232,1302,3.0,1.5
4,2001806233,245,1.0,0.5
...,...,...,...,...
119,2001821995,477,1.0,0.5
120,2001822466,1376,3.0,1.5
121,2001823564,672,2.0,1.0
122,2001825261,1557,4.0,2.0


In [15]:
tmp_file = pd.merge(tmp,company_invoice , on ='Order ID' , how = 'outer' )
tmp_file

,Order ID,total_wt_X,d_500,weight_slab_X,AWB Code,Charged Weight,Warehouse Pincode,Customer Pincode,Zone,Type of Shipment,Billing Amount (Rs.)
0,2001806210,220,1.0,0.5,1091117221940,2.92,121003,140604,b,Forward charges,174.5
1,2001806226,480,1.0,0.5,1091117222065,0.68,121003,723146,d,Forward charges,90.2
2,2001806229,500,1.0,0.5,1091117222080,0.71,121003,421204,d,Forward charges,90.2
3,2001806232,1302,3.0,1.5,1091117222124,1.30,121003,507101,d,Forward charges,140.0
4,2001806233,245,1.0,0.5,1091117222135,0.78,121003,263139,b,Forward charges,61.3
...,...,...,...,...,...,...,...,...,...,...,...
119,2001821995,477,1.0,0.5,1091121183730,0.50,121003,342008,d,Forward charges,45.4
120,2001822466,1376,3.0,1.5,1091121305541,1.10,121003,342301,d,Forward charges,135.0
121,2001823564,672,2.0,1.0,1091121666133,0.70,121003,492001,d,Forward and RTO charges,172.8
122,2001825261,1557,4.0,2.0,1091121981575,1.60,121003,517128,d,Forward and RTO charges,345.0


### Renaming the multiple column


In [16]:
tmp_file.rename(columns = {'Charged Weight':'total_wt_cc'}, inplace = True)

tmp_file.rename(columns = {'Zone':'zone_cc'}, inplace = True)

tmp_file.rename(columns = {'Billing Amount (Rs.)':'charges_by_cc'}, inplace = True)
tmp_file

,Order ID,total_wt_X,d_500,weight_slab_X,AWB Code,total_wt_cc,Warehouse Pincode,Customer Pincode,zone_cc,Type of Shipment,charges_by_cc
0,2001806210,220,1.0,0.5,1091117221940,2.92,121003,140604,b,Forward charges,174.5
1,2001806226,480,1.0,0.5,1091117222065,0.68,121003,723146,d,Forward charges,90.2
2,2001806229,500,1.0,0.5,1091117222080,0.71,121003,421204,d,Forward charges,90.2
3,2001806232,1302,3.0,1.5,1091117222124,1.30,121003,507101,d,Forward charges,140.0
4,2001806233,245,1.0,0.5,1091117222135,0.78,121003,263139,b,Forward charges,61.3
...,...,...,...,...,...,...,...,...,...,...,...
119,2001821995,477,1.0,0.5,1091121183730,0.50,121003,342008,d,Forward charges,45.4
120,2001822466,1376,3.0,1.5,1091121305541,1.10,121003,342301,d,Forward charges,135.0
121,2001823564,672,2.0,1.0,1091121666133,0.70,121003,492001,d,Forward and RTO charges,172.8
122,2001825261,1557,4.0,2.0,1091121981575,1.60,121003,517128,d,Forward and RTO charges,345.0


### Droping the unwanted culumn


In [17]:
tmp_file.drop(["d_500"], axis =1,inplace = True)
tmp_file.drop(["Warehouse Pincode"], axis =1,inplace = True)


In [18]:
tmp_file

,Order ID,total_wt_X,weight_slab_X,AWB Code,total_wt_cc,Customer Pincode,zone_cc,Type of Shipment,charges_by_cc
0,2001806210,220,0.5,1091117221940,2.92,140604,b,Forward charges,174.5
1,2001806226,480,0.5,1091117222065,0.68,723146,d,Forward charges,90.2
2,2001806229,500,0.5,1091117222080,0.71,421204,d,Forward charges,90.2
3,2001806232,1302,1.5,1091117222124,1.30,507101,d,Forward charges,140.0
4,2001806233,245,0.5,1091117222135,0.78,263139,b,Forward charges,61.3
...,...,...,...,...,...,...,...,...,...
119,2001821995,477,0.5,1091121183730,0.50,342008,d,Forward charges,45.4
120,2001822466,1376,1.5,1091121305541,1.10,342301,d,Forward charges,135.0
121,2001823564,672,1.0,1091121666133,0.70,492001,d,Forward and RTO charges,172.8
122,2001825261,1557,2.0,1091121981575,1.60,517128,d,Forward and RTO charges,345.0


###  Conveting the total weight from cubic centimer to KG


In [19]:
tmp_file['total_wt_cc'] = tmp_file['total_wt_cc'].apply(lambda x:x*1000)
tmp_file

,Order ID,total_wt_X,weight_slab_X,AWB Code,total_wt_cc,Customer Pincode,zone_cc,Type of Shipment,charges_by_cc
0,2001806210,220,0.5,1091117221940,2920.0,140604,b,Forward charges,174.5
1,2001806226,480,0.5,1091117222065,680.0,723146,d,Forward charges,90.2
2,2001806229,500,0.5,1091117222080,710.0,421204,d,Forward charges,90.2
3,2001806232,1302,1.5,1091117222124,1300.0,507101,d,Forward charges,140.0
4,2001806233,245,0.5,1091117222135,780.0,263139,b,Forward charges,61.3
...,...,...,...,...,...,...,...,...,...
119,2001821995,477,0.5,1091121183730,500.0,342008,d,Forward charges,45.4
120,2001822466,1376,1.5,1091121305541,1100.0,342301,d,Forward charges,135.0
121,2001823564,672,1.0,1091121666133,700.0,492001,d,Forward and RTO charges,172.8
122,2001825261,1557,2.0,1091121981575,1600.0,517128,d,Forward and RTO charges,345.0


### weight slab as per cc

In [20]:
tmp_file['d_500'] = tmp_file['total_wt_cc']/500
tmp_file['d_500'] = np.ceil(tmp_file['d_500'])
tmp_file['weight_slab_cc'] = tmp_file['d_500'] * 0.5
tmp_file

,Order ID,total_wt_X,weight_slab_X,AWB Code,total_wt_cc,Customer Pincode,zone_cc,Type of Shipment,charges_by_cc,d_500,weight_slab_cc
0,2001806210,220,0.5,1091117221940,2920.0,140604,b,Forward charges,174.5,6.0,3.0
1,2001806226,480,0.5,1091117222065,680.0,723146,d,Forward charges,90.2,2.0,1.0
2,2001806229,500,0.5,1091117222080,710.0,421204,d,Forward charges,90.2,2.0,1.0
3,2001806232,1302,1.5,1091117222124,1300.0,507101,d,Forward charges,140.0,3.0,1.5
4,2001806233,245,0.5,1091117222135,780.0,263139,b,Forward charges,61.3,2.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...
119,2001821995,477,0.5,1091121183730,500.0,342008,d,Forward charges,45.4,1.0,0.5
120,2001822466,1376,1.5,1091121305541,1100.0,342301,d,Forward charges,135.0,3.0,1.5
121,2001823564,672,1.0,1091121666133,700.0,492001,d,Forward and RTO charges,172.8,2.0,1.0
122,2001825261,1557,2.0,1091121981575,1600.0,517128,d,Forward and RTO charges,345.0,4.0,2.0


### Droping the column 


In [21]:
tmp_file.drop(["d_500"], axis =1,inplace = True)
tmp_file

,Order ID,total_wt_X,weight_slab_X,AWB Code,total_wt_cc,Customer Pincode,zone_cc,Type of Shipment,charges_by_cc,weight_slab_cc
0,2001806210,220,0.5,1091117221940,2920.0,140604,b,Forward charges,174.5,3.0
1,2001806226,480,0.5,1091117222065,680.0,723146,d,Forward charges,90.2,1.0
2,2001806229,500,0.5,1091117222080,710.0,421204,d,Forward charges,90.2,1.0
3,2001806232,1302,1.5,1091117222124,1300.0,507101,d,Forward charges,140.0,1.5
4,2001806233,245,0.5,1091117222135,780.0,263139,b,Forward charges,61.3,1.0
...,...,...,...,...,...,...,...,...,...,...
119,2001821995,477,0.5,1091121183730,500.0,342008,d,Forward charges,45.4,0.5
120,2001822466,1376,1.5,1091121305541,1100.0,342301,d,Forward charges,135.0,1.5
121,2001823564,672,1.0,1091121666133,700.0,492001,d,Forward and RTO charges,172.8,1.0
122,2001825261,1557,2.0,1091121981575,1600.0,517128,d,Forward and RTO charges,345.0,2.0


In [22]:
tmp_file2 = pd.merge(tmp_file , company_zones , on = 'Customer Pincode',how="left" )
tmp_file2

,Order ID,total_wt_X,weight_slab_X,AWB Code,total_wt_cc,Customer Pincode,zone_cc,Type of Shipment,charges_by_cc,weight_slab_cc,Warehouse Pincode,zone_X
0,2001806210,220,0.5,1091117221940,2920.0,140604,b,Forward charges,174.5,3.0,121003,b
1,2001806226,480,0.5,1091117222065,680.0,723146,d,Forward charges,90.2,1.0,121003,d
2,2001806229,500,0.5,1091117222080,710.0,421204,d,Forward charges,90.2,1.0,121003,d
3,2001806232,1302,1.5,1091117222124,1300.0,507101,d,Forward charges,140.0,1.5,121003,d
4,2001806233,245,0.5,1091117222135,780.0,263139,b,Forward charges,61.3,1.0,121003,b
...,...,...,...,...,...,...,...,...,...,...,...,...
169,2001821995,477,0.5,1091121183730,500.0,342008,d,Forward charges,45.4,0.5,121003,b
170,2001822466,1376,1.5,1091121305541,1100.0,342301,d,Forward charges,135.0,1.5,121003,b
171,2001823564,672,1.0,1091121666133,700.0,492001,d,Forward and RTO charges,172.8,1.0,121003,d
172,2001825261,1557,2.0,1091121981575,1600.0,517128,d,Forward and RTO charges,345.0,2.0,121003,d


In [23]:
tmp_file2.drop(["Warehouse Pincode"], axis =1,inplace = True)
tmp_file2.drop(["Customer Pincode"], axis =1,inplace = True)
tmp_file2

,Order ID,total_wt_X,weight_slab_X,AWB Code,total_wt_cc,zone_cc,Type of Shipment,charges_by_cc,weight_slab_cc,zone_X
0,2001806210,220,0.5,1091117221940,2920.0,b,Forward charges,174.5,3.0,b
1,2001806226,480,0.5,1091117222065,680.0,d,Forward charges,90.2,1.0,d
2,2001806229,500,0.5,1091117222080,710.0,d,Forward charges,90.2,1.0,d
3,2001806232,1302,1.5,1091117222124,1300.0,d,Forward charges,140.0,1.5,d
4,2001806233,245,0.5,1091117222135,780.0,b,Forward charges,61.3,1.0,b
...,...,...,...,...,...,...,...,...,...,...
169,2001821995,477,0.5,1091121183730,500.0,d,Forward charges,45.4,0.5,b
170,2001822466,1376,1.5,1091121305541,1100.0,d,Forward charges,135.0,1.5,b
171,2001823564,672,1.0,1091121666133,700.0,d,Forward and RTO charges,172.8,1.0,d
172,2001825261,1557,2.0,1091121981575,1600.0,d,Forward and RTO charges,345.0,2.0,d


In [24]:
tmp_file2.isnull().sum()

Order ID            0
total_wt_X          0
weight_slab_X       0
AWB Code            0
total_wt_cc         0
zone_cc             0
Type of Shipment    0
charges_by_cc       0
weight_slab_cc      0
zone_X              0
dtype: int64

In [25]:
tmp_file2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 174 entries, 0 to 173
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Order ID          174 non-null    int64  
 1   total_wt_X        174 non-null    int64  
 2   weight_slab_X     174 non-null    float64
 3   AWB Code          174 non-null    int64  
 4   total_wt_cc       174 non-null    float64
 5   zone_cc           174 non-null    object 
 6   Type of Shipment  174 non-null    object 
 7   charges_by_cc     174 non-null    float64
 8   weight_slab_cc    174 non-null    float64
 9   zone_X            174 non-null    object 
dtypes: float64(4), int64(3), object(3)
memory usage: 15.0+ KB


In [26]:
list_zones = tmp_file2['zone_X'].unique()
list_zones

array(['b', 'd', 'e'], dtype=object)

In [27]:
final_df = pd.DataFrame()
final_df

""


In [28]:
value_pair = {'a':29.5,'b':33,'c':40.1,'d':45.4,'e':56.6}

In [29]:
additional_charges = {'a':23.6,'b':28.3,'c':38.9,'d':44.8,'e':55.5}

In [30]:
rto_fixed_charges = {'a':13.6,'b':20.5,'c':31.9,'d':41.3,'e':50.7}

In [31]:
rto_additional_charges = {'a':23.6,'b':28.3,'c':38.9,'d':44.8,'e':55.5}

In [32]:
for i in list_zones:
    temp3 = tmp_file2[tmp_file2['zone_X'] == i]
    temp3['fixed'] = value_pair[i]
    temp3['additional'] = np.where(temp3 ['weight_slab_X']*2 > 1,((temp3['weight_slab_X']*2)-1)*additional_charges[i],0)
    temp3['rto_fixed_charges'] =  np.where(temp3['Type of Shipment'].str.contains("RTO", case=False), rto_fixed_charges[i], 0)
    temp3['rto_additional_charges'] =  np.where((temp3 ['weight_slab_X']*2 > 1) & (temp3['Type of Shipment'].str.contains('Forward and RTO charges', case=False)),((temp3['weight_slab_X']*2)-1)*rto_additional_charges[i],0)
    final_df = pd.concat([final_df,temp3])

In [33]:
final_df.reset_index(inplace = True,drop = True)
final_df

,Order ID,total_wt_X,weight_slab_X,AWB Code,total_wt_cc,zone_cc,Type of Shipment,charges_by_cc,weight_slab_cc,zone_X,fixed,additional,rto_fixed_charges,rto_additional_charges
0,2001806210,220,0.5,1091117221940,2920.0,b,Forward charges,174.5,3.0,b,33.0,0.0,0.0,0.0
1,2001806233,245,0.5,1091117222135,780.0,b,Forward charges,61.3,1.0,b,33.0,0.0,0.0,0.0
2,2001806304,500,0.5,1091117222360,710.0,d,Forward charges,90.2,1.0,b,33.0,0.0,0.0,0.0
3,2001806304,500,0.5,1091117222360,710.0,d,Forward charges,90.2,1.0,b,33.0,0.0,0.0,0.0
4,2001806304,500,0.5,1091117222360,710.0,d,Forward charges,90.2,1.0,b,33.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
169,2001811604,721,1.0,1091118004245,800.0,b,Forward charges,67.3,1.0,e,56.6,55.5,0.0,0.0
170,2001811604,721,1.0,1091118004245,800.0,b,Forward charges,67.3,1.0,e,56.6,55.5,0.0,0.0
171,2001818390,841,1.0,1091120014461,800.0,e,Forward and RTO charges,213.5,1.0,e,56.6,55.5,50.7,55.5
172,2001819252,270,0.5,1091120352712,300.0,b,Forward charges,33.0,0.5,e,56.6,0.0,0.0,0.0


In [34]:
#  Calculating the final expected charges
final_df['expected_charges_X'] = final_df['fixed'] + final_df['additional'] + final_df['rto_fixed_charges'] + final_df['rto_additional_charges']
final_df

,Order ID,total_wt_X,weight_slab_X,AWB Code,total_wt_cc,zone_cc,Type of Shipment,charges_by_cc,weight_slab_cc,zone_X,fixed,additional,rto_fixed_charges,rto_additional_charges,expected_charges_X
0,2001806210,220,0.5,1091117221940,2920.0,b,Forward charges,174.5,3.0,b,33.0,0.0,0.0,0.0,33.0
1,2001806233,245,0.5,1091117222135,780.0,b,Forward charges,61.3,1.0,b,33.0,0.0,0.0,0.0,33.0
2,2001806304,500,0.5,1091117222360,710.0,d,Forward charges,90.2,1.0,b,33.0,0.0,0.0,0.0,33.0
3,2001806304,500,0.5,1091117222360,710.0,d,Forward charges,90.2,1.0,b,33.0,0.0,0.0,0.0,33.0
4,2001806304,500,0.5,1091117222360,710.0,d,Forward charges,90.2,1.0,b,33.0,0.0,0.0,0.0,33.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
169,2001811604,721,1.0,1091118004245,800.0,b,Forward charges,67.3,1.0,e,56.6,55.5,0.0,0.0,112.1
170,2001811604,721,1.0,1091118004245,800.0,b,Forward charges,67.3,1.0,e,56.6,55.5,0.0,0.0,112.1
171,2001818390,841,1.0,1091120014461,800.0,e,Forward and RTO charges,213.5,1.0,e,56.6,55.5,50.7,55.5,218.3
172,2001819252,270,0.5,1091120352712,300.0,b,Forward charges,33.0,0.5,e,56.6,0.0,0.0,0.0,56.6


In [35]:
#  Caluculating the difference between expected charges and actual chages
final_df['difference_x_cc'] = round(final_df['expected_charges_X'] - final_df['charges_by_cc'],2)
final_df

,Order ID,total_wt_X,weight_slab_X,AWB Code,total_wt_cc,zone_cc,Type of Shipment,charges_by_cc,weight_slab_cc,zone_X,fixed,additional,rto_fixed_charges,rto_additional_charges,expected_charges_X,difference_x_cc
0,2001806210,220,0.5,1091117221940,2920.0,b,Forward charges,174.5,3.0,b,33.0,0.0,0.0,0.0,33.0,-141.5
1,2001806233,245,0.5,1091117222135,780.0,b,Forward charges,61.3,1.0,b,33.0,0.0,0.0,0.0,33.0,-28.3
2,2001806304,500,0.5,1091117222360,710.0,d,Forward charges,90.2,1.0,b,33.0,0.0,0.0,0.0,33.0,-57.2
3,2001806304,500,0.5,1091117222360,710.0,d,Forward charges,90.2,1.0,b,33.0,0.0,0.0,0.0,33.0,-57.2
4,2001806304,500,0.5,1091117222360,710.0,d,Forward charges,90.2,1.0,b,33.0,0.0,0.0,0.0,33.0,-57.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
169,2001811604,721,1.0,1091118004245,800.0,b,Forward charges,67.3,1.0,e,56.6,55.5,0.0,0.0,112.1,44.8
170,2001811604,721,1.0,1091118004245,800.0,b,Forward charges,67.3,1.0,e,56.6,55.5,0.0,0.0,112.1,44.8
171,2001818390,841,1.0,1091120014461,800.0,e,Forward and RTO charges,213.5,1.0,e,56.6,55.5,50.7,55.5,218.3,4.8
172,2001819252,270,0.5,1091120352712,300.0,b,Forward charges,33.0,0.5,e,56.6,0.0,0.0,0.0,56.6,23.6


###  Droping down the unwanted column in the dataset


In [36]:
final_df = final_df.drop(columns = ['fixed','additional','rto_fixed_charges','rto_additional_charges'])


In [37]:
df = final_df[['Order ID','AWB Code','total_wt_X','weight_slab_X','total_wt_cc','weight_slab_cc','zone_X','zone_cc','expected_charges_X','charges_by_cc','difference_x_cc']]
df

,Order ID,AWB Code,total_wt_X,weight_slab_X,total_wt_cc,weight_slab_cc,zone_X,zone_cc,expected_charges_X,charges_by_cc,difference_x_cc
0,2001806210,1091117221940,220,0.5,2920.0,3.0,b,b,33.0,174.5,-141.5
1,2001806233,1091117222135,245,0.5,780.0,1.0,b,b,33.0,61.3,-28.3
2,2001806304,1091117222360,500,0.5,710.0,1.0,b,d,33.0,90.2,-57.2
3,2001806304,1091117222360,500,0.5,710.0,1.0,b,d,33.0,90.2,-57.2
4,2001806304,1091117222360,500,0.5,710.0,1.0,b,d,33.0,90.2,-57.2
...,...,...,...,...,...,...,...,...,...,...,...
169,2001811604,1091118004245,721,1.0,800.0,1.0,e,b,112.1,67.3,44.8
170,2001811604,1091118004245,721,1.0,800.0,1.0,e,b,112.1,67.3,44.8
171,2001818390,1091120014461,841,1.0,800.0,1.0,e,e,218.3,213.5,4.8
172,2001819252,1091120352712,270,0.5,300.0,0.5,e,b,56.6,33.0,23.6


### Find out the undercharges of greater than 0


In [38]:
under_charge= df.loc[df['difference_x_cc'] > 0]
under_charge

,Order ID,AWB Code,total_wt_X,weight_slab_X,total_wt_cc,weight_slab_cc,zone_X,zone_cc,expected_charges_X,charges_by_cc,difference_x_cc
26,2001807415,1091117325094,1048,1.5,1000.0,1.0,b,b,89.6,61.3,28.3
53,2001809383,1091117614452,607,1.0,500.0,0.5,b,d,110.1,86.7,23.4
54,2001809592,1091117616121,1505,2.0,1500.0,1.5,b,b,117.9,89.6,28.3
87,2001812838,1091118547832,558,1.0,600.0,1.0,b,b,110.1,108.3,1.8
106,2001817093,1091121485824,1357,1.5,1300.0,1.5,b,b,166.7,166.1,0.6
108,2001820978,1091120922803,515,1.0,500.0,0.5,b,d,61.3,45.4,15.9
148,2001807362,1091117324394,2016,2.5,2000.0,2.0,d,d,224.6,179.8,44.8
152,2001807976,1091117327496,721,1.0,700.0,1.0,d,d,176.3,172.8,3.5
156,2001809794,1091117795531,1517,2.0,1500.0,1.5,d,d,179.8,135.0,44.8
157,2001809820,1091117795623,3080,3.5,3000.0,3.0,d,d,314.2,269.4,44.8


In [39]:
# Length of  overcharges
len(under_charge)

21

In [40]:
#  Find out the overchager of sum
under_charge.sum()

Order ID                        42038099788
AWB Code                     22913507243097
total_wt_X                            23212
weight_slab_X                          30.5
total_wt_cc                         22320.0
weight_slab_cc                         25.5
zone_X                bbbbbbddddddddeeeeeee
zone_cc               bdbbbdddddddddbbbbebb
expected_charges_X                   3526.5
charges_by_cc                        2856.9
difference_x_cc                       669.6
dtype: object

- This analyzing the dataset find out the undercharges is 669.6 RS.

### Droping the column 

In [41]:
over_charged = df.loc[df['difference_x_cc'] < 0]
over_charged

,Order ID,AWB Code,total_wt_X,weight_slab_X,total_wt_cc,weight_slab_cc,zone_X,zone_cc,expected_charges_X,charges_by_cc,difference_x_cc
0,2001806210,1091117221940,220,0.5,2920.0,3.0,b,b,33.0,174.5,-141.5
1,2001806233,1091117222135,245,0.5,780.0,1.0,b,b,33.0,61.3,-28.3
2,2001806304,1091117222360,500,0.5,710.0,1.0,b,d,33.0,90.2,-57.2
3,2001806304,1091117222360,500,0.5,710.0,1.0,b,d,33.0,90.2,-57.2
4,2001806304,1091117222360,500,0.5,710.0,1.0,b,d,33.0,90.2,-57.2
...,...,...,...,...,...,...,...,...,...,...,...
158,2001809917,1091121482593,630,1.0,600.0,1.0,d,d,176.3,179.8,-3.5
161,2001817160,1091119630264,700,1.0,700.0,1.0,d,d,176.3,181.8,-5.5
162,2001821190,1091120959015,1200,1.5,1200.0,1.5,d,d,265.9,275.9,-10.0
165,2001808295,1091117435661,245,0.5,200.0,0.5,e,e,107.3,127.3,-20.0


### Find out the undercharge of sum


In [42]:
over_charged.sum()

Order ID                                                   268242736315
AWB Code                                                146209860482502
total_wt_X                                                        83593
weight_slab_X                                                     108.0
total_wt_cc                                                    122130.0
weight_slab_cc                                                    158.0
zone_X                bbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbb...
zone_cc               bbddddddbbbbdbbbdddddddddddddddddddddddddddddd...
expected_charges_X                                               7659.5
charges_by_cc                                                   14633.3
difference_x_cc                                                 -6973.8
dtype: object

- The analyzing the dataset overcharge has 6973.8

### Fing the length


In [43]:
len(df.loc[df['difference_x_cc'] < 0])

134

In [44]:
fair_charged = df.loc[df['difference_x_cc'] == 0]
fair_charged

,Order ID,AWB Code,total_wt_X,weight_slab_X,total_wt_cc,weight_slab_cc,zone_X,zone_cc,expected_charges_X,charges_by_cc,difference_x_cc
7,2001806458,1091117223244,700,1.0,1000.0,1.0,b,b,61.3,61.3,0.0
19,2001806885,1091117229776,840,1.0,1000.0,1.0,b,b,61.3,61.3,0.0
21,2001807058,1091117323112,1168,1.5,1150.0,1.5,b,b,89.6,89.6,0.0
22,2001807058,1091117323112,1168,1.5,1150.0,1.5,b,b,89.6,89.6,0.0
29,2001807814,1091117326612,607,1.0,790.0,1.0,b,b,61.3,61.3,0.0
31,2001807956,1091117327275,1080,1.5,1080.0,1.5,b,b,89.6,89.6,0.0
34,2001808118,1091117435134,343,0.5,500.0,0.5,b,b,33.0,33.0,0.0
35,2001808118,1091117435134,343,0.5,500.0,0.5,b,b,33.0,33.0,0.0
40,2001808507,1091117436383,607,1.0,790.0,1.0,b,b,61.3,61.3,0.0
41,2001808675,1091117437050,1183,1.5,1200.0,1.5,b,b,89.6,89.6,0.0


In [45]:
len(df.loc[df['difference_x_cc'] == 0])

19

###  Find out the fair charges of sum


In [46]:
fair_charged.sum()

Order ID                      38034343428
AWB Code                   20731229352063
total_wt_X                          14718
weight_slab_X                        19.5
total_wt_cc                       16180.0
weight_slab_cc                       19.5
zone_X                bbbbbbbbbbddddddddd
zone_cc               bbbbbbbbbbddddddddd
expected_charges_X                 1436.6
charges_by_cc                      1436.6
difference_x_cc                       0.0
dtype: object

- The analyzing the dataset fair charges has 0. 

#  Converting the dataframe into the csv file


In [47]:
df.to_csv('df_file_csv', index=False)

### Conclusion Report

In this analysis, we examined the charges levied by the courier company for delivering goods ordered by customers of Company X, an e-commerce business. The aim was to verify the accuracy of the charges per order and determine if they match the expected charges calculated based on various factors.

We followed a systematic approach that involved data preparation, calculations, and comparisons. Key steps included:

### Data Preparation: 
Loaded and merged datasets containing order information, SKU master data, courier company invoices, and other relevant details.

### Calculations and Comparisons: 
calculated the total weight of orders as per X's calculations, determined weight slabs, and applied these calculations to both X's data and the courier company's data. Then estimated expected charges using X's criteria and compared them with the actual charges billed by the courier company.

### Analysis of Charges: 

#### analysis revealed the following:

### Undercharged Orders:
Identified 21 orders that were undercharged, resulting in an undercharging amount of Rs. 669.6.

### Overcharged Orders: 
Found that 134 orders were overcharged, leading to an overcharging amount of Rs. 6973.8.

### Fair Charges:
There were 19 cases where the billed charges matched the expected charges precisely.

### Final Conclusion: 
Based on our analysis, it is evident that the courier company's overcharging instances can lead to negative consequences, including customer dissatisfaction and potential damage to the company's reputation It is recommended to thoroughly review the charging methodology and ensure alignment with the calculated expectations. Addressing overcharging not only safeguards customer trust but also supports a more transparent and sustainable business model.
